<a href="https://colab.research.google.com/github/annulet/nn/blob/master/hw_5_cats_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Подключение к Google drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Вариант 2

1.Соберите модель классификатора на основе VGG16 (VGG19, ...)

2.Проведите обучение для модели на решение задачи распознавания для выборки Dog&Cat (применяем аугментацию, изменение размеров картинок + перенос обучения)

3.Сделайте выводы о том как вели себя модели в процессе обучени и что влияет на качество обучении.

In [0]:
import os, shutil

Создаем папки с изображениями для обучения, валидации и теста. Заполняем изображениями. 

In [0]:
# Для экономии ресурсов я сипользовала не все данные из набора.

In [0]:
original_train_dir = '/content/drive/My Drive/catsanddogs/train/train'
base_dir = '/content/drive/My Drive/catsanddogs/cats_and_dogs_small'
# os.mkdir(base_dir)


In [0]:
train_dir = os.path.join(base_dir, 'train')
# os.mkdir(train_dir)

In [0]:
validation_dir = os.path.join(base_dir, 'validation')
# os.mkdir(validation_dir)

In [0]:
test_dir = os.path.join(base_dir, 'test')
# os.mkdir(test_dir)

In [0]:
train_cats_dir = os.path.join(train_dir, 'cats')
# os.mkdir(train_cats_dir)

In [0]:
train_dogs_dir = os.path.join(train_dir, 'dogs')
# os.mkdir(train_dogs_dir)

In [0]:
validation_cats_dir = os.path.join(validation_dir, 'cats')
# os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
# os.mkdir(validation_dogs_dir)

In [0]:
test_cats_dir = os.path.join(test_dir, 'cats')
# os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir, 'dogs')
# os.mkdir(test_dogs_dir)

In [0]:
# fnames = ['cat.{}.jpg'.format(i) for i in range (1000)]
# for fname in fnames:
#   src = os.path.join(original_train_dir, fname)
#   dst = os.path.join(train_cats_dir, fname)
#   shutil.copyfile(src, dst)

In [0]:
# fnames = ['cat.{}.jpg'.format(i) for i in range (1000, 1500)]
# for fname in fnames:
#   src = os.path.join(original_train_dir, fname)
#   dst = os.path.join(validation_cats_dir, fname)
#   shutil.copyfile(src, dst)

In [0]:
# fnames = ['cat.{}.jpg'.format(i) for i in range (1500, 2000)]
# for fname in fnames:
#   src = os.path.join(original_train_dir, fname)
#   dst = os.path.join(test_cats_dir, fname)
#   shutil.copyfile(src, dst)

In [0]:
# fnames = ['dog.{}.jpg'.format(i) for i in range (1000)]
# for fname in fnames:
#   src = os.path.join(original_train_dir, fname)
#   dst = os.path.join(train_dogs_dir, fname)
#   shutil.copyfile(src, dst)

In [0]:
# fnames = ['dog.{}.jpg'.format(i) for i in range (1000, 1500)]
# for fname in fnames:
#   src = os.path.join(original_train_dir, fname)
#   dst = os.path.join(validation_dogs_dir, fname)
#   shutil.copyfile(src, dst)

In [0]:
# fnames = ['dog.{}.jpg'.format(i) for i in range (1500, 2000)]
# for fname in fnames:
#   src = os.path.join(original_train_dir, fname)
#   dst = os.path.join(test_dogs_dir, fname)
#   shutil.copyfile(src, dst)

Проверяем количество изображений в папках.

In [18]:
len (os. listdir(train_cats_dir))

1000

In [19]:
len (os. listdir(train_dogs_dir))

1000

In [20]:
len (os. listdir(validation_cats_dir))

500

In [21]:
len (os. listdir(validation_dogs_dir))

500

In [22]:
len (os. listdir(test_cats_dir))

500

In [23]:
len (os. listdir(test_dogs_dir))

500

Построим базовую модель

In [24]:
from keras import layers 
from keras import models

# Сверточная часть будет содержать 4 сверточных слоя, 
# 4 слоя MaxPooling2 и слой Dropout для борьбы с переобучением.

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)      

In [26]:
from keras import optimizers

# Компиляция
model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.RMSprop(lr=1e-4), 
              metrics=['acc'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Подготовим данные при помощи генераторов.

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# Масштабируем изображения, применяем аугментацию.
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=40, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)

# Тестовые изображения только масштабируем.
test_datagen = ImageDataGenerator(rescale=1./255)

In [29]:
# Обучающие данные
# Приводим к одному размеру, создаем бинарные метки.
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(150, 150), 
                                                    batch_size=20, 
                                                    class_mode='binary')

Found 2000 images belonging to 2 classes.


In [30]:
# Валидационные данные
# Приводим к одному размеру, создаем бинарные метки.
validation_generator = test_datagen.flow_from_directory(validation_dir, 
                                                    target_size=(150, 150), 
                                                    batch_size=20, 
                                                    class_mode='binary')

Found 1000 images belonging to 2 classes.


In [31]:
# Тестовые данные
# Приводим к одному размеру, создаем бинарные метки.
test_generator = test_datagen.flow_from_directory(test_dir, 
                                                    target_size=(150, 150), 
                                                    batch_size=20, 
                                                    class_mode='binary')

Found 1000 images belonging to 2 classes.


In [32]:
# Обучаем модель при помощи генератора пакетов
history = model.fit_generator(train_generator, 
                              steps_per_epoch=100, 
                              epochs=30, 
                              validation_data=validation_generator, 
                              validation_steps=50)




Epoch 1/30





100/100 [==============================] - 859s 9s/step - loss: 0.6929 - acc: 0.5215 - val_loss: 0.6819 - val_acc: 0.5680
Epoch 2/30
100/100 [==============================] - 18s 178ms/step - loss: 0.6843 - acc: 0.5520 - val_loss: 0.6719 - val_acc: 0.5620
Epoch 3/30
100/100 [==============================] - 18s 175ms/step - loss: 0.6690 - acc: 0.5765 - val_loss: 0.6729 - val_acc: 0.5570
Epoch 4/30
100/100 [==============================] - 18s 179ms/step - loss: 0.6612 - acc: 0.5895 - val_loss: 0.6578 - val_acc: 0.5960
Epoch 5/30
100/100 [==============================] - 18s 180ms/step - loss: 0.6445 - acc: 0.6175 - val_loss: 0.6268 - val_acc: 0.6400
Epoch 6/30
100/100 [==============================] - 18s 180ms/step - loss: 0.6395 - acc: 0.6175 - val_loss: 0.6143 - val_acc: 0.6470
Epoch 7/30
100/100 [==============================] - 18s 178ms/step - loss: 0.6190 - acc: 0.6525 - val_loss: 0.6017 - val_acc: 0.6640
Epoch 8/30
100/100 [=============================

In [33]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

test acc: 0.7439999997615814


In [0]:
# Сохранение модели
# model.save('cats_and_dogs_small_1.h5')

In [0]:
from keras.applications import VGG16

In [38]:
# Загружаем сеть VGG16 без части, которая отвечает за классификацию
base_model = VGG16(weights ='imagenet', 
                  include_top=False, 
                  input_shape=(150, 150, 3))

base_model.summary()

58892288/58889256 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
______________________________________________________________

In [0]:
# Добавляем полносвязный классификатор поверх сверточной основы.
model = models.Sequential() 
model.add(base_model) 
model.add(layers.Flatten()) 
model.add(layers.Dense(256, activation='relu')) 
model.add(layers.Dense(1, activation='sigmoid'))

In [40]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Замораживаем сеть VGG16 перед компиляцией
base_model.trainable = False

In [0]:
# Компиляция
model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.RMSprop(lr=1e-4), 
              metrics=['acc'])

In [43]:
history = model.fit_generator(train_generator, 
                              steps_per_epoch=100, 
                              epochs=30, 
                              validation_data=validation_generator, 
                              validation_steps=50)

Epoch 1/30
100/100 [==============================] - 21s 206ms/step - loss: 0.5361 - acc: 0.7245 - val_loss: 0.3069 - val_acc: 0.8690
Epoch 2/30
100/100 [==============================] - 18s 182ms/step - loss: 0.4169 - acc: 0.8050 - val_loss: 0.2725 - val_acc: 0.8870
Epoch 3/30
100/100 [==============================] - 18s 182ms/step - loss: 0.3963 - acc: 0.8145 - val_loss: 0.2768 - val_acc: 0.8850
Epoch 4/30
100/100 [==============================] - 18s 182ms/step - loss: 0.3595 - acc: 0.8415 - val_loss: 0.3179 - val_acc: 0.8610
Epoch 5/30
100/100 [==============================] - 19s 185ms/step - loss: 0.3523 - acc: 0.8455 - val_loss: 0.2601 - val_acc: 0.8920
Epoch 6/30
100/100 [==============================] - 18s 181ms/step - loss: 0.3497 - acc: 0.8475 - val_loss: 0.3118 - val_acc: 0.8620
Epoch 7/30
100/100 [==============================] - 18s 183ms/step - loss: 0.3396 - acc: 0.8405 - val_loss: 0.2567 - val_acc: 0.8940
Epoch 8/30
100/100 [==============================] - 1

In [44]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

test acc: 0.8939999938011169


Использование предварительно обученной нейронной сети значительно улучшило качество на тесте по сравнению с моделью с нуля, хотя для обучения использовалось всего 2000 образцов. Таким образом, этот способ построения модели оказался очень эффективным.